In [4]:
import csv
import random
from datetime import datetime, timedelta

#######################################
# Helper Functions and Bug Window Generation
#######################################

def generate_bug_windows(num_windows, min_duration, max_duration, period_start, period_end):
    """Generate a list of bug windows (start, end) within the given period."""
    windows = []
    total_seconds = int((period_end - period_start).total_seconds())
    for _ in range(num_windows):
        start_offset = random.randint(0, total_seconds)
        win_start = period_start + timedelta(seconds=start_offset)
        duration = random.randint(min_duration, max_duration)
        win_end = win_start + timedelta(seconds=duration)
        if win_end > period_end:
            win_end = period_end
        windows.append((win_start, win_end))
    return windows

def format_window(win):
    """Format a bug window tuple as a string."""
    return f"{win[0].strftime('%Y-%m-%d %H:%M:%S')} - {win[1].strftime('%Y-%m-%d %H:%M:%S')}"

def random_date_range(start, end):
    """Return a random datetime between start and end."""
    delta = end - start
    random_seconds = random.randrange(int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

#######################################
# Simulation Time Ranges
#######################################
# January (training) period:
jan_start = datetime(2025, 1, 1, 8, 0, 0)
jan_end = datetime(2025, 1, 31, 23, 59, 59)

# February (CV) period:
feb_start = datetime(2025, 2, 1, 0, 0, 0)
feb_end   = datetime(2025, 2, 5, 23, 59, 59)

#######################################
# Pre-generate Bug Windows for January
#######################################
# AP outages: 4 windows (10-30 minutes)
ap_windows = generate_bug_windows(4, 600, 1800, jan_start, jan_end)
# Video outages: 8 windows (15-29 minutes)
video_windows = generate_bug_windows(8, 900, 1740, jan_start, jan_end)
# Audio outages: 7 windows (1-3 hours)
audio_windows = generate_bug_windows(7, 3600, 10800, jan_start, jan_end)
# Twitter login outage: 1 window of 7 hours
twitter_window = generate_bug_windows(1, 25200, 25200, jan_start, jan_end)[0]

# For training, we pick the first window from each as representative:
ap_window = ap_windows[0]
video_window = video_windows[0]
audio_window = audio_windows[0]
twitter_bug_window = twitter_window

#######################################
# Pre-define Bug Windows for February (CV)
#######################################
cnn_window = (datetime(2025, 2, 1, 2, 0, 0), datetime(2025, 2, 1, 6, 0, 0))
feb_video_window = (datetime(2025, 2, 1, 8, 0, 0), datetime(2025, 2, 1, 8, 50, 0))
google_window = (datetime(2025, 2, 1, 10, 0, 0), datetime(2025, 2, 1, 16, 0, 0))

#######################################
# Event Definitions for Android and iOS
#######################################
# For DD and AS we include the corresponding screen class names.
event_definitions_android = {
    "Splash": {"dd": ["SplashScreenActivity displayed"], "as": ["SplashScreenActivity impression recorded"], "be": []},
    "LoginPage": {"dd": ["LoginActivity displayed"], "as": ["LoginActivity impression recorded"], "be": []},
    "LoginAttempt": {"dd": ["User submitted credentials in LoginActivity"], "as": ["Login button clicked in LoginActivity"], "be": ["Authentication request received", "Credentials verification started"]},
    "LoginResultSuccess": {"dd": ["LoginActivity: Login successful"], "as": ["Login success recorded from LoginActivity"], "be": ["User authenticated", "Session token generated"]},
    "LoginResultFailure": {"dd": ["LoginActivity: Login failed"], "as": ["Login failure recorded from LoginActivity"], "be": ["Authentication error", "Invalid credentials"]},
    "Home": {"dd": ["HomeScreenActivity displayed"], "as": ["HomeScreenActivity impression recorded"], "be": ["Home content requested", "Fetching home articles", "Home service responded"]},
    "ArticleText": {"dd": ["ArticleTextActivity opened"], "as": ["ArticleTextActivity impression recorded"], "be": ["Text article details fetched", "Text content loaded"]},
    "ArticleAudio": {"dd": ["ArticleAudioActivity opened"], "as": ["ArticleAudioActivity impression recorded"], "be": ["Audio content fetched", "Streaming audio started"]},
    "ArticleVideo": {"dd": ["ArticleVideoActivity opened"], "as": ["ArticleVideoActivity impression recorded"], "be": ["Video content fetched", "Streaming video delivered"]},
    "ArticleError": {"dd": ["ArticleErrorActivity: Failed to load article"], "as": ["ArticleErrorActivity impression recorded"], "be": ["Article fetch error", "Fallback to cached article"]},
    "VideoArticleError": {"dd": ["VideoViewActivity: Failed to load the video"], "as": ["VideoViewActivity error impression recorded"], "be": ["Failed to deliver the frames for the video"]},
    "AudioArticleError": {"dd": ["AudioViewActivity: Failed to load the audio"], "as": ["AudioViewActivityr error impression recorded"], "be": ["Failed to deliver the audio"]},
    "Search": {"dd": ["SearchActivity displayed"], "as": ["SearchActivity impression recorded"], "be": ["Search request processed", "Search results returned"]},
    "Share": {"dd": ["ShareActivity: User clicked share"], "as": ["Share event recorded from ShareActivity"], "be": ["Share request processed"]},
    "Exit": {"dd": ["User exited app from LastScreenActivity"], "as": ["Exit event recorded"], "be": []}
}

event_definitions_ios = {
    "Splash": {"dd": ["SplashScreenViewController displayed"], "as": ["SplashScreenViewController impression recorded"], "be": []},
    "LoginPage": {"dd": ["LoginViewController displayed"], "as": ["LoginViewController impression recorded"], "be": []},
    "LoginAttempt": {"dd": ["User submitted credentials in LoginViewController"], "as": ["Login button clicked in LoginViewController"], "be": ["Authentication request received", "Credentials verification started"]},
    "LoginResultSuccess": {"dd": ["LoginViewController: Login successful"], "as": ["Login success recorded from LoginViewController"], "be": ["User authenticated", "Session token generated"]},
    "LoginResultFailure": {"dd": ["LoginViewController: Login failed"], "as": ["Login failure recorded from LoginViewController"], "be": ["Authentication error", "Invalid credentials"]},
    "Home": {"dd": ["HomeScreenViewController displayed"], "as": ["HomeScreenViewController impression recorded"], "be": ["Home content requested", "Fetching home articles", "Home service responded"]},
    "ArticleText": {"dd": ["ArticleTextViewController opened"], "as": ["ArticleTextViewController impression recorded"], "be": ["Text article details fetched", "Text content loaded"]},
    "ArticleAudio": {"dd": ["ArticleAudioViewController opened"], "as": ["ArticleAudioViewController impression recorded"], "be": ["Audio content fetched", "Streaming audio started"]},
    "ArticleVideo": {"dd": ["ArticleVideoViewController opened"], "as": ["ArticleVideoViewController impression recorded"], "be": ["Video content fetched", "Streaming video delivered"]},
    "ArticleError": {"dd": ["ArticleErrorViewController: Failed to load article"], "as": ["ArticleErrorViewController impression recorded"], "be": ["Article fetch error", "Fallback to cached article"]},
    "VideoArticleError": {"dd": ["VideoViewController: Failed to load the video"], "as": ["VideoViewController error impression recorded"], "be": ["Failed to deliver the frames for the video"]},
    "AudioArticleError": {"dd": ["AudioViewController: Failed to load the audio"], "as": ["AudioViewController error impression recorded"], "be": ["Failed to deliver the audio"]},
    "Search": {"dd": ["SearchViewController displayed"], "as": ["SearchViewController impression recorded"], "be": ["Search request processed", "Search results returned"]},
    "Share": {"dd": ["ShareViewController: User clicked share"], "as": ["Share event recorded from ShareViewController"], "be": ["Share request processed"]},
    "Exit": {"dd": ["User exited app from LastScreenViewController"], "as": ["Exit event recorded"], "be": []}
}

#######################################
# Global Lists to Hold Logs
#######################################
jan_dd_logs, jan_be_logs, jan_as_logs = [], [], []
feb_dd_logs, feb_be_logs, feb_as_logs = [], [], []

# We'll use these for simulation sessions.
article_sources = ["wapo", "nyt", "ap", "bbc", "cnn"]
backend_services = ["LoginService", "AuthService", "HomeService", "ArticleService", "MediaService", "SearchService", "ShareService"]

#######################################
# Utility Function to Add a Log Entry
#######################################
def add_log(log_list, system, timestamp, session_id, event_idx, event_name, message, platform=None, event_type=None, article_id="", corr_id=""):
    entry = {
        "timestamp": timestamp.strftime("%Y-%m-%d %H:%M:%S"),
        "session_id": session_id,
        "event_idx": event_idx,
        "event_name": event_name,
        "log_message": message
    }
    if platform:
        entry["platform"] = platform
    if event_type:
        entry["event_type"] = event_type
    if article_id:
        entry["article_id"] = article_id
    if corr_id:
        entry["correlation_id"] = corr_id
    log_list.append(entry)

#######################################
# Simulation Function for One Session
#######################################
def simulate_session(is_jan, session_id, session_start, event_defs, bug_windows):
    # For login events, choose a login method.
    def choose_login_method():
        return random.choices(["Standard", "Twitter", "Google"], weights=[20,40,40], k=1)[0]
    
    events = []
    events.append({"event": "Splash"})
    login_success = False
    login_method = "Guest"
    if random.random() < 0.8:
        events.append({"event": "LoginPage"})
        login_method = choose_login_method()
        events.append({"event": "LoginAttempt", "login_method": login_method})
        login_success = (random.random() < 0.85)
        # For Twitter outage bug in Jan:
        if login_method == "Twitter" and is_jan:
            tw_start, tw_end = bug_windows.get("Twitter", (None, None))
            if tw_start and tw_start <= session_start <= tw_end:
                login_success = False
        events.append({"event": "LoginResultSuccess" if login_success else "LoginResultFailure", "login_method": login_method})
        if not login_success:
            events.append({"event": "Exit"})
    else:
        # reinforce guest login
        login_method = "Guest"
    
    if (login_success or login_method == "Guest"):
        events.append({"event": "Home"})
        num_articles = random.randint(0, 5)
        for _ in range(num_articles):
            art_event = random.choices(["ArticleText", "ArticleAudio", "ArticleVideo"], weights=[50,20,20], k=1)[0]
            events.append({"event": art_event})
            if random.random() < 0.3:
                events.append({"event": "Share"})
        if random.random() < 0.25:
            events.append({"event": "Search"})
        events.append({"event": "Exit"})
    
    current_time = session_start
    event_counter = 0
    for evt in events:
        event_counter += 1
        current_time += timedelta(seconds=random.randint(5, 60))
        ev_name = evt["event"]
        bug_override = None  # to hold bug type if injected
        src = random.choice(article_sources) # get the source first before doing anything else
        
        # --- January Bug Injections ---
        if is_jan and current_time <= jan_end:
            # AP outage: for Article events with source "ap" (all types of articles failed)
            if ev_name in ["ArticleText", "ArticleAudio", "ArticleVideo"]:
                if src == "ap":
                    for (ws, we) in bug_windows.get("AP", []):
                        if ws <= current_time <= we:
                            if ev_name == "ArticleText":
                                ev_name = "ArticleError"
                            elif ev_name == "ArticleAudio":
                                ev_name = "AudioArticleError"
                            else :
                                ev_name = "VideoArticleError"
                            bug_override = "AP"
                            break
            # Video outage: if event is ArticleVideo, with 50% chance
            if ev_name == "ArticleVideo":
                for (ws, we) in bug_windows.get("Video", []):
                    if ws <= current_time <= we and random.random() < 0.5:
                        ev_name = "VideoArticleError"
                        bug_override = "Video"
                        break
            # Audio outage: if event is ArticleAudio, force failure
            if ev_name == "ArticleAudio":
                for (ws, we) in bug_windows.get("Audio", []):
                    if ws <= current_time <= we:
                        ev_name = "AudioArticleError"
                        bug_override = "Audio"
                        break
            # Twitter login outage: force failure if login_method is Twitter
            if login_method == "Twitter":
                tw_start, tw_end = bug_windows.get("Twitter", (None, None))
                if tw_start and tw_start <= current_time <= tw_end:
                    if ev_name == "LoginResultSuccess":
                        ev_name = "LoginResultFailure"
                        bug_override = "Twitter"
        
        # --- February Bug Injections ---
        if (not is_jan) and (feb_start <= current_time <= feb_end):
            if ev_name in ["ArticleText", "ArticleAudio", "ArticleVideo"]:
                if src == "cnn":
                    cnn_start, cnn_end = bug_windows.get("CNN", (None, None))
                    if cnn_start and cnn_start <= current_time <= cnn_end:
                        ev_name = "ArticleError"
                        bug_override = "CNN"
            if ev_name == "ArticleVideo":
                fv_start, fv_end = bug_windows.get("Video", (None, None))
                if fv_start and fv_start <= current_time <= fv_end:
                    ev_name = "ArticleError"
                    bug_override = "VideoFeb"
            if (platform == "Android") and ("login_method" in evt) and (evt["login_method"]=="Google"):
                g_start, g_end = bug_windows.get("Google", (None, None))
                if g_start and g_start <= current_time <= g_end:
                    if ev_name == "LoginResultSuccess":
                        ev_name = "LoginResultFailure"
                        bug_override = "Google"
        
        final_event = ev_name
        
        # Generate correlation ID if event has BE call.
        if event_defs.get(final_event, {}).get("be"):
            corr_id = f"CORR-{session_id}-{event_counter}"
        else:
            corr_id = ""
        
        # For article events, assign an article ID.
        article_id = ""
        if final_event in ["ArticleText", "ArticleAudio", "ArticleVideo", "ArticleError", "VideoArticleErro", "AudioArticleError"]:
            art_src = src
            article_id = f"{art_src}{random.randint(10000000, 99999999)}"
        
        # Set default log messages.
        dd_msg = event_defs.get(final_event, {}).get("dd", [""])[0]
        as_msg = event_defs.get(final_event, {}).get("as", [""])[0]
        if event_defs.get(final_event, {}).get("be"):
            be_msg = random.choice(event_defs.get(final_event, {}).get("be", [""]))
        else:
            be_msg = ""
        
        # If a bug was injected, override messages accordingly.
        if bug_override:
            if bug_override == "AP":
                dd_msg = "ArticleErrorActivity: AP outage – article not loaded"
                as_msg = "ArticleErrorActivity: AP outage impression recorded"
                be_msg = f"External API for AP articles not responding; https://ap.com/v1/headlines/{article_id} returned 400"
            elif bug_override == "Video":
                dd_msg = "ArticleErrorActivity: Video outage – streaming library overloaded"
                as_msg = "ArticleErrorActivity: Video outage impression recorded"
                be_msg = f"Video streaming library overloaded; failed to serve video https://mynewsclips.stream/{article_id}"
            elif bug_override == "Audio":
                dd_msg = "ArticleErrorActivity: Audio outage – AI TTS service down"
                as_msg = "ArticleErrorActivity: Audio outage impression recorded"
                be_msg = f"AI Text-to-Speech service down; audio article failed https://mynewsaudiostream.stream/{article_id} has returned Internal Server Error (500) due to TTS service outage"
            elif bug_override == "Twitter":
                dd_msg = "LoginActivity: Twitter login outage – API key failure"
                as_msg = "Login failure recorded: Twitter API key failure"
                be_msg = f"Twitter login API key failure; authentication error: Exceeded Twitter API rate limit"
            elif bug_override == "CNN":
                dd_msg = "ArticleErrorActivity: CNN outage – article not loaded"
                as_msg = "ArticleErrorActivity: CNN outage impression recorded"
                be_msg = f"External API for CNN articles not responding; https://cnn.com/v1/headlines/{article_id} returned 400"
            elif bug_override == "VideoFeb":
                dd_msg = "ArticleErrorActivity: Video outage – streaming failure (Feb)"
                as_msg = "ArticleErrorActivity: Video outage impression recorded (Feb)"
                be_msg = f"Video streaming library overloaded; failed to serve video https://mynewsclips.stream/{article_id}"
            elif bug_override == "Google":
                dd_msg = "LoginActivity: Google login outage – OAuth SDK failure"
                as_msg = "Login failure recorded: Google login outage"
                be_msg = f"Google OAuth API failure; authentication error: Client SDK is deprecated, please upgrade to the version >= 6.7.8"
            if event_defs.get(final_event, {}).get("be"):
                num_lines = random.randint(1, 3)
                be_lines = []
                for i in range(num_lines):
                    line_time = (current_time + timedelta(seconds=i * 3)).strftime("%Y-%m-%d %H:%M:%S")
                    be_lines.append(f"{line_time} | {be_msg}")
                be_msg = "\n".join(be_lines)
        
        # Add logs to the corresponding global lists.
        add_log(current_dd_logs, "DD", current_time, session_id, event_counter, final_event, dd_msg, platform=platform, article_id=article_id, corr_id=corr_id)
        as_event_type = "click" if final_event in ["LoginAttempt", "Share"] else "impression"
        add_log(current_as_logs, "AS", current_time, session_id, event_counter, final_event, as_msg, event_type=as_event_type, article_id=article_id)
        if event_defs.get(final_event, {}).get("be"):
            add_log(current_be_logs, "BE", current_time, session_id, event_counter, final_event, be_msg, article_id=article_id, corr_id=corr_id)

#######################################
# Simulate Sessions for January and February
#######################################
NUM_SESSIONS_JAN = 10000  # January sessions
NUM_SESSIONS_FEB = 5000   # February sessions

# Simulate January sessions
current_dd_logs = jan_dd_logs
current_be_logs = jan_be_logs
current_as_logs = jan_as_logs
for sess in range(1, NUM_SESSIONS_JAN + 1):
    session_start = random_date_range(jan_start, jan_end)
    platform = random.choice(["Android", "iOS"])
    sess_prefix = "sess-A" if platform == "Android" else "sess-I"
    session_id = f"{sess_prefix}{10000 + sess}"
    event_defs = event_definitions_android if platform == "Android" else event_definitions_ios
    simulate_session(True, session_id, session_start, event_defs, {
        "AP": ap_windows,
        "Video": video_windows,
        "Audio": audio_windows,
        "Twitter": twitter_window
    })

# Simulate February sessions
current_dd_logs = feb_dd_logs
current_be_logs = feb_be_logs
current_as_logs = feb_as_logs
for sess in range(1, NUM_SESSIONS_FEB + 1):
    session_start = random_date_range(feb_start, feb_end)
    platform = random.choice(["Android", "iOS"])
    sess_prefix = "sess-A" if platform == "Android" else "sess-I"
    session_id = f"{sess_prefix}{10000 + sess}"
    event_defs = event_definitions_android if platform == "Android" else event_definitions_ios
    feb_bug_windows = {
        "CNN": cnn_window,
        "Video": feb_video_window,
        "Google": google_window
    }
    simulate_session(False, session_id, session_start, event_defs, feb_bug_windows)

#######################################
# Write Combined Log CSV Files
#######################################
all_dd_logs = jan_dd_logs + feb_dd_logs
all_be_logs = jan_be_logs + feb_be_logs
all_as_logs = jan_as_logs + feb_as_logs

with open("datadog_mobile_logs.csv", "w", newline="") as f:
    fieldnames = ["timestamp", "session_id", "event_idx", "event_name", "platform", "log_message", "article_id", "correlation_id"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(all_dd_logs)

with open("backend_logs.csv", "w", newline="") as f:
    fieldnames = ["timestamp", "session_id", "event_idx", "event_name", "correlation_id", "service", "log_message", "article_id"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(all_be_logs)

with open("analytics_logs.csv", "w", newline="") as f:
    fieldnames = ["timestamp", "session_id", "event_idx", "event_name", "event_type", "log_message", "article_id"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(all_as_logs)

print("Generated log CSV files:")
print(f"  - DataDog logs: {len(all_dd_logs)} rows")
print(f"  - Backend logs: {len(all_be_logs)} rows")
print(f"  - Analytics logs: {len(all_as_logs)} rows")

#######################################
# Generate Jira Tickets CSV Files
#######################################
# Training Tickets (January bugs) – include RCA and Time_Window
training_tickets = [
    {
        "Issue_ID": "JIRA-TRAIN-001",
        "Project": "Live Issues",
        "Summary": "AP Article Outage",
        "Description": "Users reported that articles from the Associated Press were not loading during a specific window.",
        "RCA": "The external AP API experienced downtime during several 10-30 minute windows, causing AP articles to fail.",
        "Bug_Type": "AP Outage",
        "Time_Window": [format_window(i) for i in ap_windows]
    },
    {
        "Issue_ID": "JIRA-TRAIN-002",
        "Project": "Live Issues",
        "Summary": "Video Streaming Failure",
        "Description": "Intermittent failures in video article streams were observed during peak times.",
        "RCA": "Overload of the video streaming library caused failures in serving video content during 15-29 minute windows.",
        "Bug_Type": "Video Outage",
        "Time_Window": [format_window(i) for i in video_windows]
    },
    {
        "Issue_ID": "JIRA-TRAIN-003",
        "Project": "Live Issues",
        "Summary": "Audio Article Outage",
        "Description": "Multiple sessions showed that audio articles failed to load over extended periods.",
        "RCA": "Outages in the AI Text-to-Speech service led to audio article failures during 1-3 hour windows.",
        "Bug_Type": "Audio Outage",
        "Time_Window": [format_window(i) for i in audio_windows]
    },
    {
        "Issue_ID": "JIRA-TRAIN-004",
        "Project": "Live Issues",
        "Summary": "Twitter Login Outage",
        "Description": "Sessions using Twitter login encountered failures for a prolonged period.",
        "RCA": "A 7-hour Twitter API key failure resulted in complete outage for Twitter login functionality.",
        "Bug_Type": "Twitter Login Outage",
        "Time_Window": format_window(twitter_bug_window)
    }
]
for ticket in training_tickets:
    created_date = random_date_range(datetime(2025, 2, 1), datetime(2025, 2, 2))
    ticket["Created_Date"] = created_date.strftime("%Y-%m-%d %H:%M:%S")

# CV Tickets (February bugs) – RCA omitted
cv_tickets = [
    {
        "Issue_ID": "JIRA-CV-001",
        "Project": "Live Issues",
        "Summary": "CNN Article Outage",
        "Description": "Several sessions reported that CNN articles were not loading during a specific window.",
        "Bug_Type": "CNN Outage",
        "Time_Window": format_window(cnn_window)
    },
    {
        "Issue_ID": "JIRA-CV-002",
        "Project": "Live Issues",
        "Summary": "Video Article Outage (CV)",
        "Description": "A transient failure in video article streams was observed during a 50-minute window.",
        "Bug_Type": "Video Outage",
        "Time_Window": format_window(feb_video_window)
    },
    {
        "Issue_ID": "JIRA-CV-003",
        "Project": "Live Issues",
        "Summary": "Google Login Outage (Android)",
        "Description": "Android sessions using Google login consistently failed during a 6-hour window.",
        "Bug_Type": "Google Login Outage",
        "Time_Window": format_window(google_window)
    }
]
for ticket in cv_tickets:
    created_date = random_date_range(datetime(2025, 2, 6), datetime(2025, 2, 7))
    ticket["Created_Date"] = created_date.strftime("%Y-%m-%d %H:%M:%S")

with open("jira_tickets_training.csv", "w", newline="") as f:
    fieldnames = ["Issue_ID", "Project", "Summary", "Description", "RCA", "Created_Date", "Bug_Type", "Time_Window"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(training_tickets)

with open("jira_tickets_cv.csv", "w", newline="") as f:
    fieldnames = ["Issue_ID", "Project", "Summary", "Description", "Created_Date", "Bug_Type", "Time_Window"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(cv_tickets)

print("Generated Jira tickets CSV files for Training and CV data.")

Generated log CSV files:
  - DataDog logs: 125750 rows
  - Backend logs: 83766 rows
  - Analytics logs: 125750 rows
Generated Jira tickets CSV files for Training and CV data.
